In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [72]:
species = "horse"
te_file = 'horse/'+species+'_244_te.txt'
sv_file= 'horse/'+species+'_insertion_244.txt'
output_file = 'horse/'+species+'_annotated_244.txt'
output_dups_file = 'horse/'+species+'_final_244.txt'

In [52]:
df_te = pd.read_csv(te_file, sep='\t', lineterminator='\n')
df_te.columns = ['chromA','startA','endA','subfam','fam', 'chromB','startB','endB','species']
print(df_te.head())

  chromA   startA     endA subfam      fam chromB   startB     endB    species
0      1  2163380  2163484  L1ME2  LINE/L1      1  2163415  2163415      akhal
1      1  2163380  2163484  L1ME2  LINE/L1      1  2163395  2163395    arabian
2      1  2163380  2163484  L1ME2  LINE/L1      1  2163415  2163415    quarter
3      1  2163380  2163484  L1ME2  LINE/L1      1  2163415  2163415    quarter
4      1  2163380  2163484  L1ME2  LINE/L1      1  2163415  2163415  mongolian


In [58]:
with open(sv_file) as f:
    with open(output_file, 'w') as r:
        lines = f.readlines()
        for line in lines:
            line = line.split()
            if ">" in line[0]:
                seq = ""
                t = line[0].split(":")
                chrom = t[0].strip(">")
                start = t[1].split("-")[0]
                end = t[1].split("-")[0]
                spec = t[1].split("?")[1]
                df_query = df_te.loc[(df_te['chromB'] == chrom) & (df_te['startB'] == int(start))]
                if len(df_query) > 0:
                    for i,row in df_query.iterrows():
                        subfam = row['subfam']
                        fam = row['fam']
                        break
                    line[0] += ";"+fam+","+subfam
                header = line[0]
            else:
                seq = line[0]
            if seq != '':
                r.write(header + '\n')
                r.write(seq + '\n')
print('done')

done


In [91]:
### check for duplicates ####
chroms = []
starts = []
ends = []
seqs = []
species = []
families = []
subfamilies = []
with open(output_file) as f:
    lines = f.readlines()
    for line in lines:
        line = line.split()
        if ">" in line[0]:
            seq = ""
            t = line[0].split(":")
            chrom = t[0].strip(">")
            start = t[1].split("-")[0]
            end = t[1].split("-")[0]
            spec = t[1].split("?")[1]
            if ";" in t[1]:
                spec = spec.split(";")[0]
                fams = t[1].split(";")[1]
                families.append(fams.split(",")[0])
                subfamilies.append(fams.split(",")[1])
            else:
                families.append("NA")
                subfamilies.append("NA")
            chroms.append(chrom)
            starts.append(start)
            ends.append(end)
            species.append(spec)
        else:
            seq = line[0]
            seqs.append(seq)
df = pd.DataFrame()
df['chrom'] = chroms
df['start'] = starts
df['end'] = ends
df['species'] = species
df['fam'] = families
df['subfam'] = subfamilies
df['seq'] = seqs
df = df.drop_duplicates(subset='start', keep='first')
df = df[df['fam']!='LINE/L1']
print(df.head())         

  chrom      start        end     species            fam   subfam  \
0     1   43943650   43943650  przewalski             NA       NA   
1     1   71425667   71425667  przewalski  DNA/TcMar-Tc2  Kanga1a   
3     1  156704553  156704553  przewalski             NA       NA   
4     1  182960158  182960158  przewalski             NA       NA   
6     2   60216026   60216026  przewalski             NA       NA   

                                                 seq  
0  GGGCTGGCCCCGTGGCCGAGTGGTTAAGTTCGCGCGCTCCGCTGCA...  
1  GGGGCTGGCCCCGTGGCCGAGCGGTTAAGTTCGCGCGCTCCGCTGC...  
3  GTATTTGGAGTTTGGTTTTGGTGTTGTTTCTTGGGCTCCAATCAGT...  
4  GGGGCTGGCCCCGTGGCCAAGTGGTTAAGTTCGCGCGCTCCGCTGC...  
6  CTCAACCACAGAACTTTTTTTTTTTTTTTTTAAAGATTTTATTTTT...  


In [92]:
with open(output_dups_file, 'w') as f:
    for i,row in df.iterrows():
        coords = '>'+str(row['chrom'])+':'+str(row['start'])+'-'+str(row['end']) \
            +"?"+str(row['species'])+";"+str(row['fam'])+','+str(row['subfam'])
        #coords = str(row['chrom'])+'\t'+str(row['start'])+'\t'+str(row['end'])
        f.write(coords)
        f.write('\n')
        f.write(row['seq'])
        f.write('\n')